In [ ]:
!pip3 install --upgrade pip
!pip3 install boto3
!pip3 install pyyaml

In [17]:
from functools import reduce
import os
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
import yaml
from datetime import datetime, time, timedelta
import json
from pprint import pprint
from pprint import PrettyPrinter
from statistics import mean
import sys

In [ ]:
print(sys.version_info)
print(sys.version_info[:3])

In [ ]:
config = yaml.load(open('config.yaml', 'r'))
#print(config)
aws_region = config['aws']['regions'][0]
account = config['aws']['accounts'][1]['name']
profile = config['aws']['accounts'][1]['profile']
account_number = config['aws']['accounts'][1]['account-number']

print(("%s - %s - %s - %s" % (str(account_number),account,profile,aws_region)))

In [20]:
def err_chk(fnCall, errormsg):
        errStr = "{} returned error: {}".format(fnCall, e)
        print(errStr)
        return

In [ ]:
session = boto3.session.Session(profile_name=profile,region_name=aws_region)
print(session)

In [ ]:
s3Client = session.client('s3', config=Config(signature_version='s3v4'))
print(s3Client)

In [ ]:
buckets = s3Client.list_buckets()
print(("%s buckets" % (len(buckets.get('Buckets')))))


In [ ]:
pp = PrettyPrinter() #indent=1)
pp.pprint(buckets)

In [ ]:
location = s3Client.get_bucket_location(Bucket='<BUCKET_NAME>')
pp.pprint(location)

In [26]:
denied = []

for bucket in buckets.get('Buckets'):
    bucketName = bucket.get('Name')
    try:
        head = s3Client.head_bucket(Bucket=bucketName)
    except Exception as inst:
        #print(type(inst))    # the exception instance
        #print(inst.args[0])     # arguments stored in .args
        if "403" in inst.args[0]:
            denied.append(bucketName)
            
print(denied)

[]


In [ ]:
try:
    tags = s3Client.get_bucket_tagging(Bucket='<BUCKET_NAME>')
    print(tags)
    pp.pprint(tags)
except ClientError as e:
    err_chk("Tags could not be retrieved", e)
    #return

In [ ]:
try:
    policy = s3Client.get_bucket_policy(Bucket='<BUCKET_NAME>')
    pp.pprint(policy)
except ClientError as e:
    err_chk("Policy could not be retrieved", e)

In [ ]:
try:
    acl = s3Client.get_bucket_acl(Bucket='<BUCKET_NAME>')
    pp.pprint(acl)
except ClientError as e:
    err_chk("ACL could not be retrieved",e)


In [ ]:
try:
    notifications = s3Client.get_bucket_notification(Bucket='<BUCKET_NAME>')
    pp.pprint(notifications)
except ClientError as e:
    err_chk("Notifications could not be retrieved",e)

In [39]:
def tag_value_exists(tagSet, tagKey, tagValue):
    for tag in tagSet.get('TagSet'):
        if tag.get('Key') == tagKey:
            if tag.get('Value') == tagValue:
                return True
        
    return False

In [ ]:
tagged = {}
failed = []

for bucket in buckets.get('Buckets'):
    #print(bucket)
    bucketName = bucket.get('Name')
    #print(bucketName)

    
    #access = s3Client.head_bucket(Bucket=bucketName)
    #print(access)
    
    if not bucketName in denied:
        try:
            tagSet = s3Client.get_bucket_tagging(Bucket=bucketName)
            if tag_value_exists(tagSet, 'TAG_KEY', 'TAG_VALUE'):
                print(bucketName)
                tagged[bucketName] = tagSet
        except:
            failed.append(bucketName)

print(tagged)
print(failed)

In [ ]:
pp.pprint(tagged)

In [ ]:
try:
    objects = s3Client.list_objects(Bucket='<BUCKET_NAME>')
    print(objects)
    contents = objects.get('Contents')
    if contents != None:
        print(contents)
        print(len(contents))
    pp.pprint(objects)
except ClientError as e:
    err_chk("ACL could not be retrieved",e)